In [ ]:
"""
Author: Pengbo Li
Created: 2024-10-22
Version: 1.0

Description: 
    Simulation of the Aluminum cylinder LVDT design.
    Check the LVDT performance of the Aluminum cylinder, magnet core, and the combination of both.

Model:  
- Aluminum cylinder, including change the inner diameter, outer diameter, and length.
- magnet core, including change the diameter and length.
- air coil + magnet core, including different combinations of the magnet core and air coil.

"""
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from modules import simulator, geometry, coreConfig, dataHandler


In [ ]:
path = "../data/"
filename = 'LVDT_alucld_d0806_l12_magcore_d0804_l04.h5'

# # Model: aluminum cylinder + magnetic core
# path = "../data/"
# # filename_template = 'LVDT_alucld_d{outer_str}d{inner_str}_l12_magcore_d{mag_diam_str}_l{mag_len_str}.h5'
# filename_template = 'LVDT_magcore_d{mag_diam_str}_l{mag_len_str}.h5'
# outer_diam = [8]

# mag_length = [4]
# mag_diam = [8]

# pairs = generate_pairs(outer_diam,min_thickness=2,max_thickness=2)

# for pair in pairs:
#     outer_str = str(pair[0]).zfill(2)
#     inner_str = str(pair[1]).zfill(2)
#     mag_diam_str = str(mag_diam[0]).zfill(2)
#     mag_len_str = str(mag_length[0]).zfill(2)
#     filename = filename_template.format(
#         # outer_str=outer_str,
#         # inner_str=inner_str,
#         mag_diam_str=mag_diam_str,
#         mag_len_str=mag_len_str
#     )

#     print("data will be saved to: ", path + filename)



simulation_freq = 10000
simulation_amplitude = 0.02

Magnet_geo = geometry.def_core_geo(8, 4, 'N40')
Alu_geo  = geometry.def_cylinder_geo(6, 8, 12, 'Aluminum, 6061-T6')
MiddleCoil_geo = geometry.def_coil_geo("100um", 0.1, 0, 16,  3, 18, 0)
OuterCoil_geo  = geometry.def_coil_geo("100um", 0.1, 0, 16,  3, 18, 16)

MC_circuit = simulator.def_circuit_prop("middlecoil", 0, 0)
OC_upper_circuit = simulator.def_circuit_prop("outercoil_upper", simulation_freq, simulation_amplitude)
OC_lower_circuit = simulator.def_circuit_prop("outercoil_lower", simulation_freq, -simulation_amplitude)


simulator.def_femm_problem(signal_frequency = simulation_freq)
simulator.build_air_geometry("Outside", 10)

m_label = simulator.build_core_geometry(Magnet_geo, 1)
alu_label = simulator.build_cylinder_geometry(Alu_geo, 2)
mc_label = simulator.build_coil_geometry(MiddleCoil_geo, MC_circuit, 3,customized_material=True)
oc_upper_label = simulator.build_coil_geometry(OuterCoil_geo, OC_upper_circuit, 4, customized_material=True )
oc_lower_label = simulator.build_coil_geometry(OuterCoil_geo, OC_lower_circuit, 5, customized_material=True, reverse=True)

print("Magnet label: ", m_label)
print("Alu cylinder label: ", alu_label)
print("Middle coil label: ", mc_label)
print("Outer coil upper label: ", oc_upper_label)
print("Outer coil lower label: ", oc_lower_label)

cc_config = coreConfig.moving_config(-5, 1,10)
lvdt_voltage= simulator.def_lvdt_voltage(cc_config['steps'])
sim_results = simulator.lvdt_simulation(moving_parts_label  = [1],
                            CC_config           = cc_config,  
                            lvdt_voltage        = lvdt_voltage, 
                            OC_upper_circuit    = OC_upper_circuit,
                            OC_lower_circuit    = OC_lower_circuit,
                            MC_circuit          =MC_circuit,
                            CC_circuit          =None)

dataHandler.save_data(sim_results, path + filename)

print("data saved to: ", path + filename)